# Career Copilot — Демонстрация pipeline обработки данных

Этот ноутбук демонстрирует полный цикл обработки данных в системе Career Copilot:  
от загрузки справочников до генерации персонального плана развития.

**Структура:**
1. Загрузка и исследование данных
2. Нормализация навыков (pymorphy3 vs stemming)
3. Парсинг резюме (PDF → навыки через GPT)
4. RAG pipeline (эмбеддинги + Qdrant)
5. Сценарии и gap-анализ
6. Генерация плана развития 70/20/10

---
## 1. Загрузка и исследование данных

Система работает с тремя JSON-справочниками:  
- **clean_skills.json** — навыки с привязкой к профессиям и уровнями владения  
- **atlas_params_clean.json** — метакомпетенции (параметры атласа) по грейдам  
- **skill_synonyms.json** — словарь синонимов для нормализации ввода

In [ ]:
import json
import sys
from pathlib import Path
from collections import Counter

PROJECT_DIR = Path(".").resolve()
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

with open("data/clean_skills.json", "r", encoding="utf-8") as f:
    skills_data = json.load(f)

with open("data/atlas_params_clean.json", "r", encoding="utf-8") as f:
    atlas_data = json.load(f)

with open("data/skill_synonyms.json", "r", encoding="utf-8") as f:
    synonyms_data = json.load(f)

print(f"Навыков:          {len(skills_data)}")
print(f"Параметров атласа: {len(atlas_data)}")
print(f"Синонимов:         {len(synonyms_data)}")

### 1.1. Структура записи навыка

Каждый навык содержит описания трёх уровней владения (Basic / Proficiency / Advanced)  
и примеры задач на развитие для каждого уровня.

In [ ]:
example_skill = skills_data[1]
print(f"Навык:      {example_skill.get('Навык')}")
print(f"Профессия:  {example_skill.get('Профессия (лист)')}")
print(f"Категория:  {example_skill.get('Категория')}")
print(f"Свойства:   {example_skill.get('Свойства')}")
print()
print("--- Уровни владения ---")
for level in ["Basic", "Proficiency", "Advanced"]:
    key = f"Skill level \\\\ Индикатор - {level}"
    desc = example_skill.get(key, "")
    print(f"\n{level}: {desc}")

### 1.2. Профессии и распределение навыков

In [ ]:
profession_counts = Counter()
category_counts = Counter()

for s in skills_data:
    prof = s.get("Профессия (лист)") or s.get("Привязка к профессии", "")
    cat = s.get("Категория", "")
    if prof:
        profession_counts[prof] += 1
    if cat:
        category_counts[cat] += 1

print(f"Профессий: {len(profession_counts)}")
print(f"Категорий навыков: {len(category_counts)}")
print()
print("--- Навыков по профессиям ---")
for prof, count in profession_counts.most_common():
    short = prof.replace("Скиллсет ", "")
    print(f"  {short:45s} {count:3d} навыков")

In [ ]:
print("--- Топ-15 категорий навыков ---")
for cat, count in category_counts.most_common(15):
    print(f"  {cat:45s} {count:3d}")

### 1.3. Параметры атласа (метакомпетенции)

7 параметров, общих для всех профессий.  
Для каждого определены ожидания по 5 грейдам: Младший → Эксперт.

In [ ]:
for param in atlas_data:
    name = param.get("Параметр", "")
    desc = param.get("Описание", "")
    print(f"\n{'='*70}")
    print(f"Параметр: {name}")
    print(f"Описание: {desc[:120]}..." if len(desc) > 120 else f"Описание: {desc}")
    print()
    for grade in ["Младший", "Специалист", "Старший", "Ведущий", "Эксперт"]:
        text = param.get(grade, "")
        short = text[:100] + "..." if len(text) > 100 else text
        print(f"  {grade:12s} │ {short}")

### 1.4. Словарь синонимов

In [ ]:
print(f"Всего синонимов: {len(synonyms_data)}")
print()
print("--- Примеры маппинга ---")
for synonym, canonical in synonyms_data.items():
    print(f"  '{synonym}' → '{canonical}'")

### 1.5. DataLoader — унифицированный доступ к данным

Модуль `data_loader.py` строит индексы для быстрого доступа:  
маппинг внутренних названий профессий ↔ отображаемых, получение требований роли по грейду.

In [ ]:
from data_loader import DataLoader

data = DataLoader()

all_roles = data.get_all_roles()
print(f"Доступные профессии ({len(all_roles)}):")
for role in all_roles:
    print(f"  • {role}")

In [ ]:
role = "Менеджер продукта"
role_skills = data.get_skills_for_role(role)
print(f"Навыки для '{role}' ({len(role_skills)}):")
for s in role_skills:
    print(f"  • {s}")

In [ ]:
internal_name = data.get_internal_role_name(role)
requirements = data.get_role_requirements(internal_name, "Middle")

atlas_params = [p.get("Параметр") for p in atlas_data]
skill_reqs = {k: v for k, v in requirements.items() if k not in atlas_params}
param_reqs = {k: v for k, v in requirements.items() if k in atlas_params}

print(f"Требования для '{role}' на уровне Middle:")
print(f"  Навыков:    {len(skill_reqs)} (все на уровне {list(skill_reqs.values())[0] if skill_reqs else '?'} — Proficiency)")
print(f"  Параметров: {len(param_reqs)} (все на уровне {list(param_reqs.values())[0] if param_reqs else '?'} — Специалист)")

---
## 2. Нормализация навыков

Пользователь может ввести навык в произвольной форме.  
Система приводит ввод к каноническому названию через цепочку:  
**точное совпадение → опечатки → лемматизация (pymorphy3) → синонимы**

### 2.1. Лемматизация vs стемминг

Стемминг (NLTK Snowball) грубо отсекает окончания, а лемматизация (pymorphy3)  
приводит слово к словарной форме с учётом морфологии.

In [ ]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

test_words = [
    "разработка", "разработчик", "разработчиков", "разработки",
    "тестирование", "тестировщик", "тестировщики",
    "управление", "управлять", "управляет",
    "аналитик", "аналитика", "аналитический",
    "коммуникация", "коммуникации", "коммуникативный",
]

try:
    from nltk.stem.snowball import SnowballStemmer
    stemmer = SnowballStemmer("russian")
    has_stemmer = True
except Exception:
    has_stemmer = False

print(f"{'Слово':25s} │ {'Лемма (pymorphy3)':25s}", end="")
if has_stemmer:
    print(f" │ {'Стем (Snowball)':25s}", end="")
print(f" │ Совпадение?")
print("─" * 110)

for word in test_words:
    lemma = morph.parse(word)[0].normal_form
    if has_stemmer:
        stem = stemmer.stem(word)
        match = "✓" if lemma == morph.parse(test_words[0])[0].normal_form else " "
        print(f"{word:25s} │ {lemma:25s} │ {stem:25s} │ {match}")
    else:
        print(f"{word:25s} │ {lemma:25s} │")

### 2.2. Демонстрация normalize_for_search

Функция нормализации: lowercase → исправление опечаток → лемматизация по словам.

In [ ]:
from skill_normalizer import normalize_for_search

test_inputs = [
    "Анализ данных",
    "анализа данных",
    "МАШИННОЕ ОБУЧЕНИЕ",
    "машинного обучения",
    "Управление проектами",
    "управления проектом",
    "питон",
    "Python",
    "Data Science",
    "Написание кода",
]

print(f"{'Ввод':35s} │ {'Нормализованная форма':35s}")
print("─" * 75)
for inp in test_inputs:
    norm = normalize_for_search(inp)
    print(f"{inp:35s} │ {norm:35s}")

### 2.3. Разрешение в каноническое название (resolve_to_canonical)

Цепочка ответственности: точное совпадение → нормализованная форма → перебор ключей.

In [ ]:
from skill_normalizer import resolve_to_canonical, get_canonical_skills_set

canonical_set = get_canonical_skills_set()
print(f"Канонических навыков в справочнике: {len(canonical_set)}")
print()

test_queries = [
    "питон",
    "Пайтон",
    "python 3",
    "эксель",
    "машинное обучение",
    "machine learning",
    "управление проектами",
    "командная работа",
    "программирование",
    "несуществующий навык",
]

print(f"{'Ввод пользователя':30s} │ {'Канонический навык':30s}")
print("─" * 65)
for q in test_queries:
    result = resolve_to_canonical(q, canonical_set)
    print(f"{q:30s} │ {result or '— не найден':30s}")

---
## 3. Парсинг резюме

Двухэтапный процесс:
1. **pypdf** — извлечение текста из PDF  
2. **GPT-4o** — извлечение навыков с оценкой уровня (JSON mode)

> Для демонстрации показываем структуру промпта и формат ответа.  
> Реальный вызов требует OPENAI_API_KEY.

In [ ]:
from resume_parser import ResumeParser
from config import Config

parser = ResumeParser()
print(f"OpenAI API доступен: {'Да' if parser.client else 'Нет (OPENAI_API_KEY не задан)'}")
print(f"Модель: {Config.RESUME_PARSER_MODEL}")
print(f"Лимит текста: {Config.RESUME_TEXT_MAX_CHARS} символов")

In [ ]:
sample_skills = list(data.skills_map.keys())[:20]
skills_str = ", ".join(sample_skills)

system_prompt = f"""
Ты HR-аналитик. Извлеки навыки из резюме, сопоставь со списком: [{skills_str}, ...].
Оцени уровень 1-3 (1-Junior, 2-Middle, 3-Senior).
Верни только JSON: {{"skills": [{{"name": "название", "level": число}}]}}
""".strip()

print("--- Системный промпт для GPT-4o ---")
print(system_prompt)
print()
print("--- Пример ожидаемого ответа ---")
print(json.dumps({
    "skills": [
        {"name": "SQL, YQL", "level": 2},
        {"name": "Коммуникация", "level": 3},
        {"name": "Управление проектами", "level": 2},
    ]
}, ensure_ascii=False, indent=2))

### Конвертация уровней

GPT возвращает уровни 1-3.  
Фронтенд использует шкалу 0–2 (шаг 0.5).  
Маппинг: GPT 1→1, 2→1.5, 3→2.

In [ ]:
level_mapping = {1: 1, 2: 1.5, 3: 2}

gpt_response = [
    {"name": "SQL, YQL", "level": 2},
    {"name": "Коммуникация", "level": 3},
    {"name": "DataLens", "level": 1},
]

level_labels = {0: 'Нет навыка', 0.5: 'Начальный', 1: 'Базовый', 1.5: 'Продвинутый', 2: 'Эксперт'}

print(f"{'Навык':25s} │ {'GPT (1-3)':10s} │ {'Frontend (0-2)':15s} │ {'Уровень'}")
print("─" * 75)

for s in gpt_response:
    fe_level = level_mapping.get(s["level"], 1)
    label = level_labels.get(fe_level, str(fe_level))
    print(f"{s['name']:25s} │ {s['level']:10d} │ {fe_level:15.1f} │ {label}")

---
## 4. RAG Pipeline

RAG (Retrieval-Augmented Generation) обогащает ответы семантически релевантным контекстом.

**Компоненты:**
- **Sentence-Transformers** — модель `paraphrase-multilingual-MiniLM-L12-v2` (384-dim)
- **Qdrant** — векторная БД с косинусной метрикой

> Для демонстрации покажем процесс генерации эмбеддингов.  
> Реальный поиск в Qdrant требует QDRANT_URL и QDRANT_API_KEY.

In [ ]:
print(f"Модель эмбеддингов: {Config.EMBED_MODEL_NAME}")
print(f"Qdrant URL: {'настроен' if Config.QDRANT_URL else 'не задан'}")
print(f"RAG коллекция: {Config.RAG_COLLECTION_NAME}")
print(f"Top-K: {Config.RAG_TOP_K}")
print(f"Score threshold: {Config.RAG_SCORE_THRESHOLD}")
print(f"Skill map threshold: {Config.SKILL_MAP_SIMILARITY_THRESHOLD}")

In [ ]:
# Демонстрация генерации эмбеддингов (загрузка модели может занять ~30 сек)
try:
    from sentence_transformers import SentenceTransformer
    import numpy as np

    model = SentenceTransformer(Config.EMBED_MODEL_NAME)

    texts = [
        "Управление проектами в IT",
        "Project management",
        "Машинное обучение",
        "Machine learning",
        "SQL-запросы и базы данных",
        "Написание кода на Python",
    ]

    embeddings = model.encode(texts)

    print(f"Размерность эмбеддинга: {embeddings.shape[1]}")
    print(f"Кол-во текстов: {embeddings.shape[0]}")
    print()

    # Косинусная близость между парами
    from numpy.linalg import norm
    def cosine_sim(a, b):
        return np.dot(a, b) / (norm(a) * norm(b))

    print("--- Косинусная близость ---")
    pairs = [(0,1), (2,3), (0,2), (4,5), (0,4)]
    for i, j in pairs:
        sim = cosine_sim(embeddings[i], embeddings[j])
        print(f"  '{texts[i]}' ↔ '{texts[j]}'")
        print(f"  → similarity = {sim:.3f}")
        print()

except ImportError:
    print("sentence-transformers не установлен. Пропускаем демонстрацию эмбеддингов.")

### 4.1. Структура данных в Qdrant

Каждый навык индексируется как документ с payload:
```json
{
  "type": "skill",
  "name": "SQL, YQL",
  "profession": "ProductManager",
  "text": "SQL, YQL — Пишет простые SELECT-запросы..."
}
```

При поиске: запрос → эмбеддинг → cosine search в Qdrant → top-K результатов.

---
## 5. Сценарии и gap-анализ

Система поддерживает три сценария. Для каждого выполняется gap-анализ:  
сопоставление текущих навыков пользователя с требованиями целевой роли.

### 5.1. Моделирование пользователя

Зададим профиль: Product Manager, Middle, с набором навыков.

In [ ]:
from data_loader import GRADE_TO_PARAM_ORDINAL

user_profile = {
    "profession": "Менеджер продукта",
    "grade": "Middle",
    "skills": {
        "SQL, YQL": 2,
        "DataLens": 1,
        "Трекер": 2,
        "Коммуникация": 2,
        "Управление проектами": 1,
        "A/B-эксперименты": 1,
        "Формулирование задач": 2,
    }
}

# Добавляем атлас-параметры (автоматически по грейду)
user_skills = dict(user_profile["skills"])
current_ordinal = GRADE_TO_PARAM_ORDINAL.get(user_profile["grade"], 2)
for param in atlas_data:
    param_name = param.get("Параметр")
    if param_name not in user_skills:
        user_skills[param_name] = current_ordinal

print(f"Профессия: {user_profile['profession']}")
print(f"Грейд:     {user_profile['grade']}")
print(f"Навыков:   {len(user_profile['skills'])}")
print(f"+ атлас-параметров: {len(atlas_data)} (уровень {current_ordinal})")
print(f"Итого ключей: {len(user_skills)}")

### 5.2. Сценарий «Следующий грейд» (Middle → Senior)

Загружаем требования Senior, сравниваем с текущим профилем.

In [ ]:
from scenario_handler import ScenarioHandler
from gap_analyzer import GapAnalyzer, level_display

scenarios = ScenarioHandler(data)
analyzer = GapAnalyzer()

internal = data.get_internal_role_name(user_profile["profession"])
target_reqs, role_name = scenarios.next_grade(internal, user_profile["grade"], user_skills)

print(f"Цель: {role_name}")
print(f"Требований в целевом грейде: {len(target_reqs)}")

In [ ]:
atlas_param_names = [p.get("Параметр") for p in atlas_data]
atlas_map = {p.get("Параметр"): p for p in atlas_data}

structured = analyzer.analyze_structured(
    user_skills, target_reqs, atlas_param_names, atlas_map
)

print(f"Соответствие: {structured['match_percent']}%")
print(f"Разрывов (атлас):  {len(structured['atlas_gaps'])}")
print(f"Разрывов (навыки): {len(structured['skill_gaps'])}")
print(f"Сильные стороны:   {len(structured['atlas_strong']) + len(structured['skill_strong'])}")

In [ ]:
print("\n--- Разрывы по параметрам атласа ---")
print(f"{'Параметр':35s} │ {'Текущий':12s} │ {'Требуемый':12s} │ {'Δ':3s} │ Приоритет")
print("─" * 85)
for g in structured["atlas_gaps"]:
    curr_name = level_display(g["current"], is_atlas=True)
    req_name = level_display(g["required"], is_atlas=True)
    prio = "🔴 Критич." if g["priority"] == 1 else "🟡 Умерен." if g["priority"] == 2 else "🟢 Незнач."
    print(f"{g['name']:35s} │ {curr_name:12s} │ {req_name:12s} │ {g['delta']:3d} │ {prio}")

print("\n--- Разрывы по навыкам ---")
print(f"{'Навык':35s} │ {'Текущий':12s} │ {'Требуемый':12s} │ {'Δ':3s} │ Приоритет")
print("─" * 85)
for g in structured["skill_gaps"][:10]:
    curr_name = level_display(g["current"], is_atlas=False)
    req_name = level_display(g["required"], is_atlas=False)
    prio = "🔴 Критич." if g["priority"] == 1 else "🟡 Умерен." if g["priority"] == 2 else "🟢 Незнач."
    print(f"{g['name']:35s} │ {curr_name:12s} │ {req_name:12s} │ {g['delta']:3d} │ {prio}")

if len(structured["skill_gaps"]) > 10:
    print(f"... и ещё {len(structured['skill_gaps']) - 10} навыков с разрывами")

In [ ]:
print("--- Сильные стороны ---")
for s in structured["skill_strong"]:
    lvl = level_display(s["level"], is_atlas=False)
    print(f"  ✅ {s['name']} — {lvl}")
for s in structured["atlas_strong"]:
    lvl = level_display(s["level"], is_atlas=True)
    print(f"  ✅ {s['name']} — {lvl}")

### 5.3. Сценарий «Смена профессии»

Сравним навыки Product Manager с требованиями Аналитика-разработчика.

In [ ]:
target_role = "Аналитик-разработчик"
target_internal = data.get_internal_role_name(target_role)
target_reqs_switch = data.get_role_requirements(target_internal, "Middle")

switch_result = analyzer.analyze_structured(
    user_skills, target_reqs_switch, atlas_param_names, atlas_map
)

print(f"Переход: {user_profile['profession']} → {target_role}")
print(f"Соответствие: {switch_result['match_percent']}%")
print(f"Совпадающих навыков: {len(switch_result['skill_strong'])}")
print(f"Недостающих навыков: {len([g for g in switch_result['skill_gaps'] if g['current'] == 0])}")
print(f"Частичных (нужно прокачать): {len([g for g in switch_result['skill_gaps'] if g['current'] > 0])}")

### 5.4. Сценарий «Исследование возможностей»

Перебираем все профессии, считаем процент пересечения навыков,  
категоризируем: ближайшие (≥15%), смежные (5–15%), дальние (<5%).

In [ ]:
opps = scenarios.explore_opportunities(user_skills)

print(f"Найдено возможностей: {len(opps)}")
print()
print(f"{'Роль':45s} │ {'Совпадение':12s} │ Категория")
print("─" * 80)
for opp in sorted(opps, key=lambda x: -x.get("match", 0))[:15]:
    match = opp.get("match", 0)
    role_name_opp = opp.get("role", "")
    if match >= 15:
        cat = "🟢 Ближайшая"
    elif match >= 5:
        cat = "🟡 Смежная"
    else:
        cat = "🔴 Дальняя"
    print(f"{role_name_opp:45s} │ {match:10.1f}% │ {cat}")

---
## 6. Генерация плана развития

Финальный этап: `output_formatter` собирает диагностику в Markdown,  
затем `plan_generator` отправляет её в GPT-4o для генерации плана 70/20/10.

**Модель 70/20/10:**
- **70%** — обучение через реальные задачи на работе
- **20%** — взаимодействие: менторство, обратная связь, code review
- **10%** — формальное обучение: курсы, книги, тренинги

In [ ]:
from output_formatter import OutputFormatter

formatter = OutputFormatter(data)

# Формируем markdown-диагностику (шаг 1)
grade_sequence = ["Junior", "Middle", "Senior", "Lead", "Expert"]
current_index = grade_sequence.index(user_profile["grade"])
target_grade = grade_sequence[min(current_index + 1, len(grade_sequence) - 1)]

md = formatter.format_next_grade(
    structured,
    role_name,
    user_profile["profession"],
    current_grade=user_profile["grade"],
    target_grade=target_grade,
    profession_internal=internal,
)

# Показываем первые 2000 символов результата
preview = md[:2000]
if len(md) > 2000:
    preview += "\n\n... [обрезано для демонстрации] ..."

print(f"Длина полного отчёта: {len(md)} символов")
print(f"{'='*70}")
print(preview)

### 6.1. Промпт для GPT-4o

Показываем структуру промпта, который отправляется в LLM для генерации плана.

In [ ]:
from plan_generator import PlanGenerator, _compress_step1, MAX_STEP1_CHARS, MAX_TOKENS_RESPONSE

compressed = _compress_step1(md)

print(f"Оригинальная диагностика: {len(md)} символов")
print(f"Сжатая для LLM:           {len(compressed)} символов (лимит: {MAX_STEP1_CHARS})")
print(f"Max tokens ответа:        {MAX_TOKENS_RESPONSE}")
print(f"Температура:              0.3")
print()

prompt_template = f"""Ты карьерный консультант. На основе диагностики составь персональный план развития.

Цель: {role_name}
Тип сценария: Следующий грейд

Диагностика:
[сжатая диагностика — {len(compressed)} символов]

Требования к ответу:
1. Развитие через реальные задачи (70%)
2. Взаимодействие и обратная связь (20%)
3. Курсы и тренинги (10%)
4. Метрики прогресса и чекпоинты (4 / 8 / 12 недель)"""

print("--- Структура промпта ---")
print(prompt_template)

### 6.2. Генерация плана (требует OPENAI_API_KEY)

In [ ]:
gen = PlanGenerator()

if gen.client:
    plan = gen.generate_plan_702010(
        scenario_type="Следующий грейд",
        step1_markdown=md,
        target_name=role_name,
        context=f"Текущий грейд: {user_profile['grade']}, целевой: {target_grade}",
    )
    print("--- Сгенерированный план ---")
    print(plan[:3000])
    if len(plan) > 3000:
        print(f"\n... [всего {len(plan)} символов]")
else:
    print("OPENAI_API_KEY не задан — показываем fallback-план:")
    print()
    print(gen._fallback_plan(role_name))

---
## 7. Итоговый pipeline

Полный цикл обработки одного запроса, как его выполняет API:

In [ ]:
print("▸ Шаг 1: Ввод пользователя")
print(f"  Профессия: {user_profile['profession']}")
print(f"  Грейд: {user_profile['grade']}")
print(f"  Навыков: {len(user_profile['skills'])}")
print(f"  Сценарий: Следующий грейд")

print("\n▸ Шаг 2: Нормализация навыков")
from skill_normalizer import resolve_to_canonical, get_canonical_skills_set
cs = get_canonical_skills_set()
for name in user_profile["skills"]:
    canonical = resolve_to_canonical(name, cs)
    if canonical and canonical != name:
        print(f"  '{name}' → '{canonical}'")
    else:
        print(f"  '{name}' — без изменений")

print(f"\n▸ Шаг 3: Загрузка требований ({role_name})")
print(f"  Навыков: {len(skill_reqs)}, Параметров: {len(param_reqs)}")

print(f"\n▸ Шаг 4: Gap-анализ")
print(f"  Соответствие: {structured['match_percent']}%")
print(f"  Разрывов: {len(structured['atlas_gaps'])} (атлас) + {len(structured['skill_gaps'])} (навыки)")

print(f"\n▸ Шаг 5: Форматирование диагностики")
print(f"  Markdown: {len(md)} символов")

print(f"\n▸ Шаг 6: Генерация плана 70/20/10")
if gen.client:
    print(f"  GPT-4o: план сгенерирован ({len(plan)} символов)")
else:
    print(f"  Fallback (OPENAI_API_KEY не задан)")

print(f"\n▸ Шаг 7: Отправка на фронтенд")
print(f"  JSON: {{markdown: '...{len(md)} chars...', role_titles: null}}")
print("\n✅ Pipeline завершён")